## **Respuestas del examen final**



### **Pregunta 1**

#### Generación de datos sintéticos

##### **Diseño del conjunto de datos**

Queremos crear un conjunto de datos en $\mathbb{R}^2$ (bidimensional) en el que dos clases se solapen parcialmente. Una forma sencilla es utilizar distribuciones gaussianas (normales) para cada clase. Por ejemplo:

- **Clase 0:**  
  $$
  \mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_0, \Sigma), \quad \boldsymbol{\mu}_0 = \begin{pmatrix}0 \\ 0\end{pmatrix}
  $$
- **Clase 1:**  
  $$
  \mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_1, \Sigma), \quad \boldsymbol{\mu}_1 = \begin{pmatrix}1 \\ 1\end{pmatrix}
  $$

Usamos la misma matriz de covarianza para ambas clases, por ejemplo:
$$
\Sigma = \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix}.
$$
Dado que la distancia entre las medias es del mismo orden que la desviación estándar, se garantiza un solapamiento parcial.

##### **Pseudocódigo para la generación de datos**

```plaintext
# Número de ejemplos por clase
N0 = 100
N1 = 100

# Parámetros de la distribución
mu0 = [0, 0]
mu1 = [1, 1]
Sigma = [[1, 0], [0, 1]]  # Matriz identidad

# Inicializar listas vacías para los datos y etiquetas
datos = []
etiquetas = []

# Generar ejemplos para la clase 0
para i de 1 hasta N0:
    x = muestra_aleatoria_normal(mu0, Sigma)
    datos.agregar(x)
    etiquetas.agregar(0)

# Generar ejemplos para la clase 1
para i de 1 hasta N1:
    x = muestra_aleatoria_normal(mu1, Sigma)
    datos.agregar(x)
    etiquetas.agregar(1)
```

*Nota:* La función `muestra_aleatoria_normal(mu, Sigma)` representa una muestra de una distribución normal multivariada con media $ \mu $ y covarianza $ \Sigma $. Esta es la idea general sin utilizar librerías externas.


#### Modelo de regresión logística con regularización mixta (Elastic Net)

El modelo de regresión logística predice la probabilidad $ p $ de pertenecer a la clase 1 mediante:
$$
p = \sigma(z) = \frac{1}{1 + e^{-z}} \quad \text{con} \quad z = \mathbf{w}^\top \mathbf{x} + b.
$$

##### **Función de costo con Elastic Net**

La función de costo a minimizar es la suma del error de log-verosimilitud (log-loss) y el término de regularización elástico, que combina L1 y L2:
$$
J(\mathbf{w}, b) = \frac{1}{N} \sum_{i=1}^{N} \left[- y_i \log \sigma(z_i) - (1 - y_i) \log \bigl(1 - \sigma(z_i)\bigr)\right] + \alpha \|\mathbf{w}\|_1 + \frac{1-\alpha}{2}\|\mathbf{w}\|_2^2,
$$
donde:
- $ \|\mathbf{w}\|_1 = \sum_j |w_j| $
- $ \|\mathbf{w}\|_2^2 = \sum_j w_j^2 $
- $ \alpha \in [0,1] $ controla la contribución de cada penalización.

El primer término mide la discrepancia entre las predicciones y las etiquetas, mientras que el segundo y tercer término evitan el sobreajuste forzando algunas componentes a cero (L1) y limitando el tamaño global de los pesos (L2).


##### **Introduciendo no linealidad para romper la convexidad**

Si únicamente usamos una transformación fija de las features (por ejemplo, añadir $x_1^2$, $x_2^2$ y $x_1 x_2$) la regresión logística sigue siendo un modelo lineal en los parámetros y, por tanto, el problema de optimización es convexo.

Para que la función de costo deje de ser convexa, se debe introducir una transformación no lineal *parametrizada*, por ejemplo, utilizando una pequeña red neuronal con una capa oculta. Una opción es la siguiente:

#### Modelo no lineal (red neuronal pequeña)

1. **Capa oculta:**  
   Se calcula un vector de activación:
   $$
   \mathbf{h} = \tanh(W \mathbf{x} + \mathbf{c}),
   $$
   donde:
   - $ W $ es la matriz de pesos de la capa oculta.
   - $ \mathbf{c} $ es el vector de sesgos.
   - La función $\tanh(\cdot)$ introduce la no linealidad.

2. **Capa de salida (logística):**  
   La salida se calcula como:
   $$
   z = \mathbf{v}^\top \mathbf{h} + b, \quad \hat{y} = \sigma(z) = \frac{1}{1+e^{-z}},
   $$
   donde:
   - $ \mathbf{v} $ y $ b $ son los parámetros de la capa de salida.

3. **Función de costo con regularización mixta:**
   $$
   J(W, \mathbf{c}, \mathbf{v}, b) = \frac{1}{N}\sum_{i=1}^{N} \left[- y_i \log(\hat{y}_i) - (1-y_i) \log(1-\hat{y}_i)\right] + \alpha \left(\|W\|_1 + \|\mathbf{v}\|_1\right) + \frac{1-\alpha}{2}\left(\|W\|_2^2 + \|\mathbf{v}\|_2^2\right).
   $$

En este modelo, la función $\tanh$ y la forma en que $W$ y $\mathbf{v}$ intervienen hacen que el costo sea **no convexo** respecto a los parámetros. Es decir, la composición de la función de activación no lineal con la función de costo (incluso si el término de regularización por sí solo es convexo) produce un paisaje de error con múltiples mínimos locales y mesetas.

##### **Análisis teórico de la no convexidad**


- **Regresión logística lineal (con transformación fija):**  
  La función de costo es convexa respecto a los parámetros $ \mathbf{w} $ y $ b $ cuando el modelo es lineal en las features. Incluso al agregar regularización L1 y L2, el problema sigue siendo convexo, pues ambos términos de regularización son funciones convexas.

- **Incorporación de transformaciones parametrizadas no lineales:**  
  Cuando introducimos una capa oculta con una función de activación no lineal (como $\tanh$), la salida depende de los parámetros de $W$ y $\mathbf{c}$ de forma no lineal. Así, la composición de la función de pérdida con estas transformaciones produce un problema de optimización **no convexo**.  
  Además:
  - El término L1 introduce puntos de no diferenciabilidad (por ejemplo, en $w_j = 0$), lo que añade “esquinas” en el paisaje del costo.
  - La combinación de L1 y L2 (Elastic Net) crea una penalización mixta que, aun siendo convexa cuando se aplica a un modelo lineal, se integra en un sistema no lineal con múltiples interacciones entre parámetros.
  - La interacción entre la no linealidad del modelo (que puede tener múltiples mínimos locales y puntos silla) y la discontinuidad en la derivada del término L1 genera una función de costo complicada, en la cual los algoritmos de optimización (como gradiente descendente) pueden quedar atrapados o tener dificultades para encontrar el óptimo global.

#### Curvas de entrenamiento y dificultad en la optimización

##### **Comportamiento observado en la práctica**

Al entrenar el modelo (por ejemplo, la red neuronal pequeña con regularización Elastic Net) se pueden observar curvas de error vs. época con las siguientes características:

- **Disminución irregular del error:**  
  Debido a la presencia de múltiples mínimos locales y puntos de inflexión en la función de costo, la curva de error puede presentar "saltos" o mesetas. Esto significa que el error no decrece de forma suave y monotónica, sino que puede estancarse o incluso aumentar temporalmente antes de seguir decreciendo.

- **Sensibilidad a la tasa de aprendizaje:**  
  La combinación de L1 y L2 hace que el paisaje de la función de costo tenga zonas planas (debido al efecto "de empuje" de L1 para poner a cero ciertos parámetros) junto con regiones de fuerte curvatura (por L2). Esto exige una tasa de aprendizaje muy cuidadosamente ajustada para evitar que el algoritmo oscile o se atasque.

- **Dificultades en la convergencia:**  
  Los "picos" y "valles" en la función de costo pueden llevar a que el algoritmo se quede en óptimos locales que no son los globales, complicando el proceso de convergencia.

### Ejemplo esquemático de una curva de error

Imagina la siguiente descripción gráfica (se recomienda generar un gráfico real en una implementación práctica):

```
Error
  │          *        *
  │         * *      * *
  │        *   *    *   *
  │       *     *  *     *
  │-----*-------**-------*-----→ Épocas
```

En este esquema se observa que, en lugar de un descenso suave, el error disminuye de forma irregular, lo que es característico en escenarios no convexos.



#### **Implementaciones**

##### **1. Generación de datos sintéticos**

Utilizaremos dos distribuciones normales multivariadas en 2D para generar dos clases con cierto solapamiento. En este ejemplo:

- **Clase 0:**  
  $ \mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_0, \Sigma) $  
  con $ \boldsymbol{\mu}_0 = \begin{pmatrix} 0 \\ 0 \end{pmatrix} $
  
- **Clase 1:**  
  $ \mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_1, \Sigma) $  
  con $ \boldsymbol{\mu}_1 = \begin{pmatrix} 1 \\ 1 \end{pmatrix} $

Se utiliza la matriz de covarianza $ \Sigma = \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix} $.

El siguiente código genera 100 muestras por clase:




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_data(N0, N1):
    # Parámetros para la distribución de cada clase
    mu0 = np.array([0, 0])
    mu1 = np.array([1, 1])
    Sigma = np.array([[1, 0], [0, 1]])
    
    # Generar datos usando una distribución normal multivariada
    X0 = np.random.multivariate_normal(mu0, Sigma, N0)
    X1 = np.random.multivariate_normal(mu1, Sigma, N1)
    
    # Combinar datos y etiquetas
    X = np.vstack([X0, X1])
    y = np.hstack([np.zeros(N0), np.ones(N1)])
    return X, y

# Generamos los datos sintéticos
np.random.seed(42)
N0, N1 = 100, 100
X, y = generate_data(N0, N1)

# Visualizamos el conjunto de datos
plt.scatter(X[:N0,0], X[:N0,1], label="Clase 0", alpha=0.6)
plt.scatter(X[N0:,0], X[N0:,1], label="Clase 1", alpha=0.6)
plt.legend()
plt.title("Datos sintéticos 2D con solapamiento parcial")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()


#### Modelo de regresión logística con regularización mixta (Elastic Net)

Recordemos que en un modelo de regresión logística se calcula:

$$
p = \sigma(z) = \frac{1}{1+e^{-z}}, \quad \text{donde } z = \mathbf{w}^\top \mathbf{x} + b.
$$

La función de costo con regularización mixta es

$$
J(\mathbf{w}, b) = \frac{1}{N} \sum_{i=1}^{N} \Bigl[- y_i \log \sigma(z_i) - (1 - y_i) \log (1-\sigma(z_i))\Bigr] + \alpha \|\mathbf{w}\|_1 + \frac{1-\alpha}{2}\|\mathbf{w}\|_2^2.
$$

En el código que sigue se implementa un modelo extendido a una **pequeña red neuronal** con una capa oculta (usando $\tanh$) para introducir una transformación no lineal parametrizada. De esta forma, la composición de la función de activación con la función de costo hace que ésta sea **no convexa** respecto a los parámetros.


Se implementa una red con:
- **Capa oculta:**  
  $\mathbf{h} = \tanh(W\mathbf{x} + \mathbf{c})$
- **Capa de salida:**  
  $z = \mathbf{v}^\top \mathbf{h} + b$ y  
  $\hat{y} = \sigma(z)$

La función de costo es la log-loss más el término de regularización elastic net aplicado a $W$ y $\mathbf{v}$. Se entrena mediante gradiente descendente.



In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    # Derivada de tanh es 1 - tanh(x)^2
    return 1 - np.tanh(x)**2

def train_nn(X, y, params, alpha_reg=0.5, lr=0.01, epochs=200):
    """
    Entrena una red neuronal pequeña con una capa oculta y regularización Elastic Net.
    params: diccionario con parámetros 'W', 'c', 'v' y 'b'
    alpha_reg: peso del término L1 (L1) en la regularización; (1-alpha_reg) para L2.
    """
    N = X.shape[0]
    loss_history = []
    
    for epoch in range(epochs):
        # Inicializar gradientes acumulados
        grad_W = np.zeros_like(params['W'])
        grad_c = np.zeros_like(params['c'])
        grad_v = np.zeros_like(params['v'])
        grad_b = 0.0
        loss_epoch = 0
        
        # Procesamiento de cada muestra (modo "batch" completo)
        for i in range(N):
            x = X[i]   # Entrada (dimensión d)
            yi = y[i]
            
            # Forward pass
            z_hidden = np.dot(params['W'], x) + params['c']  # (H,)
            h = tanh(z_hidden)  # Activación de la capa oculta (H,)
            z_out = np.dot(params['v'], h) + params['b']       # Escalar
            pred = sigmoid(z_out)
            
            # Cálculo del loss (log-loss)
            loss_sample = - (yi * np.log(pred+1e-8) + (1-yi)*np.log(1-pred+1e-8))
            loss_epoch += loss_sample
            
            # Backward pass
            # Derivada de la pérdida respecto de z_out
            dL_dz_out = pred - yi   # Escalar
            
            # Gradientes para la capa de salida
            grad_v += dL_dz_out * h          # (H,)
            grad_b += dL_dz_out              # Escalar
            
            # Gradientes para la capa oculta
            dL_dh = dL_dz_out * params['v']   # (H,)
            dL_dz_hidden = dL_dh * tanh_derivative(z_hidden)  # (H,)
            
            # Gradientes para W y c
            grad_W += np.outer(dL_dz_hidden, x)  # (H, d)
            grad_c += dL_dz_hidden               # (H,)
        
        # Promediar gradientes sobre todas las muestras
        grad_W /= N
        grad_c /= N
        grad_v /= N
        grad_b /= N
        
        # Agregar gradientes de la regularización Elastic Net
        # Término L2: derivada es (1-alpha_reg)*parametro
        grad_W += (1-alpha_reg) * params['W']
        grad_v += (1-alpha_reg) * params['v']
        # Término L1: subgradiente es alpha_reg * sign(parametro)
        grad_W += alpha_reg * np.sign(params['W'])
        grad_v += alpha_reg * np.sign(params['v'])
        
        # Actualización de parámetros con tasa de aprendizaje lr
        params['W'] -= lr * grad_W
        params['c'] -= lr * grad_c
        params['v'] -= lr * grad_v
        params['b'] -= lr * grad_b
        
        # Calcular el loss promedio para la época y agregar la regularización
        loss_epoch /= N
        L1_term = alpha_reg * (np.sum(np.abs(params['W'])) + np.sum(np.abs(params['v'])))
        L2_term = (1-alpha_reg)/2 * (np.sum(params['W']**2) + np.sum(params['v']**2))
        total_loss = loss_epoch + L1_term + L2_term
        loss_history.append(total_loss)
        
        if epoch % 10 == 0:
            print(f"Epoca {epoch}, Loss: {total_loss}")
    
    return params, loss_history

# Inicialización de parámetros de la red neuronal
d = 2    # Dimensión de la entrada
H = 5    # Número de neuronas en la capa oculta

params = {
    'W': np.random.randn(H, d) * 0.1,  # Pesos de la capa oculta
    'c': np.zeros(H),                  # Sesgos de la capa oculta
    'v': np.random.randn(H) * 0.1,       # Pesos de la capa de salida
    'b': 0.0                           # Sesgo de la capa de salida
}

# Entrenamiento de la red neuronal con regularización Elastic Net
alpha_reg = 0.5  # Parámetro que mezcla L1 y L2
lr = 0.01        # Tasa de aprendizaje
epochs = 200

params, loss_history = train_nn(X, y, params, alpha_reg, lr, epochs)

# Visualización de la curva de error (loss vs. época)
plt.plot(loss_history)
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Curva de entrenamiento: Loss vs Épocas")
plt.show()



### **Pregunta 2**

####  Definición de la función de activación personalizada

Definimos la función de activación:
$$
\phi(z) = z \, \sigma(z) + \beta \cdot \sin(z)
$$
donde  
- $\sigma(z) = \frac{1}{1+e^{-z}}$ es la función sigmoide,  
- $\beta$ es un parámetro entrenable adicional.

##### **Derivadas**

**Derivada de $\phi(z)$ respecto a $z$:**

1. Para el término $z\,\sigma(z)$:  
   Utilizando la regla del producto:
   $$
   \frac{d}{dz}[z\,\sigma(z)] = \sigma(z) + z\,\sigma'(z)
   $$
   y recordando que
   $$
   \sigma'(z) = \sigma(z)\bigl(1-\sigma(z)\bigr),
   $$
   obtenemos:
   $$
   \frac{d}{dz}[z\,\sigma(z)] = \sigma(z) + z\,\sigma(z)\bigl(1-\sigma(z)\bigr).
   $$

2. Para el término $\beta \sin(z)$:  
   Al tratarse de $\beta$ constante respecto a $z$, su derivada es:
   $$
   \frac{d}{dz}[\beta \sin(z)] = \beta \cos(z).
   $$

**Por lo tanto:**
$$
\frac{\partial \phi}{\partial z} = \sigma(z) + z\,\sigma(z)(1-\sigma(z)) + \beta\,\cos(z).
$$

**Derivada de $\phi(z)$ respecto a $\beta$:**

Como $\beta$ solo aparece en el segundo término:
$$
\frac{\partial \phi}{\partial \beta} = \sin(z).
$$


#### Implementación de la red neuronal con activación personalizada

Consideraremos una red muy simple con:
- **Capa de entrada:** 2 neuronas.
- **Capa oculta:** 2 neuronas que usan la función $\phi(z)$ definida.
- **Capa de salida:** 1 neurona (usaremos activación lineal para simplificar el ejemplo).

##### **Esquema del modelo**

1. **Forward:**

   - **Capa oculta:**  
     Para una entrada $ \mathbf{x} \in \mathbb{R}^2 $,
     $$
     \mathbf{z}^{(1)} = W^{(1)}\mathbf{x} + \mathbf{b}^{(1)} \quad \text{(dimensión: 2)}
     $$
     La activación es:
     $$
     \mathbf{a}^{(1)} = \phi\bigl(\mathbf{z}^{(1)}\bigr) \quad \text{aplicada elemento a elemento}.
     $$
     
   - **Capa de salida:**  
     $$
     z^{(2)} = W^{(2)}\cdot \mathbf{a}^{(1)} + b^{(2)}
     $$
     y la salida (por ejemplo, usando activación lineal) es:
     $$
     \hat{y} = z^{(2)}.
     $$
     
   - **Función de pérdida:**  
     Usaremos el error cuadrático:
     $$
     L = \frac{1}{2}\bigl(\hat{y} - y\bigr)^2.
     $$

2. **Backward  (cálculo manual de gradientes):**

   Sea $ \delta^{(2)} = \frac{\partial L}{\partial z^{(2)}} = \hat{y} - y $.  
   Los gradientes en la capa de salida son:
   $$
   \frac{\partial L}{\partial W^{(2)}} = \delta^{(2)} \cdot \mathbf{a}^{(1)} \quad,\quad \frac{\partial L}{\partial b^{(2)}} = \delta^{(2)}.
   $$
   
   Para la capa oculta, se propaga el error:
   $$
   \delta^{(1)} = \left(W^{(2)}\right)^T \delta^{(2)} \odot \phi'(z^{(1)}),
   $$
   donde $\odot$ es el producto elemento a elemento y $\phi'(z^{(1)})$ se evalúa con la derivada definida:
   $$
   \phi'(z) = \sigma(z) + z\,\sigma(z)(1-\sigma(z)) + \beta\,\cos(z).
   $$
   
   Los gradientes respecto a los parámetros de la capa oculta son:
   $$
   \frac{\partial L}{\partial W^{(1)}} = \delta^{(1)} \cdot \mathbf{x}^T,\quad \frac{\partial L}{\partial \mathbf{b}^{(1)}} = \delta^{(1)}.
   $$
   
   **Importante:** Además, para el parámetro $\beta$ se tiene
   $$
   \frac{\partial L}{\partial \beta} = \sum_{i}\delta^{(1)}_i \cdot \frac{\partial \phi(z^{(1)}_i)}{\partial \beta} \quad \text{y recordemos que} \quad \frac{\partial \phi(z)}{\partial \beta} = \sin(z).
   $$
   En este ejemplo, se suma sobre las neuronas de la capa oculta (o se toma el promedio si se entrena con mini-batch).

A continuación se muestra un ejemplo completo con un solo dato (pasos forward y backward ) para ilustrar el proceso.

In [ ]:
import numpy as np

# Definiciones básicas
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def phi(z, beta):
    """
    Función de activación personalizada:
    phi(z) = z * sigmoid(z) + beta * sin(z)
    """
    return z * sigmoid(z) + beta * np.sin(z)

def dphi_dz(z, beta):
    """
    Derivada de phi(z) respecto a z:
    dphi/dz = sigmoid(z) + z * sigmoid(z)*(1-sigmoid(z)) + beta*cos(z)
    """
    s = sigmoid(z)
    return s + z * s * (1 - s) + beta * np.cos(z)

def dphi_dbeta(z):
    """
    Derivada de phi(z) respecto a beta:
    dphi/d(beta) = sin(z)
    """
    return np.sin(z)

# Inicialización de parámetros para la red
np.random.seed(0)
# Para la capa oculta (2 neuronas de entrada -> 2 neuronas ocultas)
W1 = np.random.randn(2, 2) * 0.1  # matriz 2x2
b1 = np.random.randn(2) * 0.1       # vector de 2
# Parámetro beta para la activación (global para ambas neuronas ocultas)
beta = 0.1

# Para la capa de salida (2 neuronas ocultas -> 1 salida)
W2 = np.random.randn(1, 2) * 0.1   # vector fila de 2
b2 = 0.0

# Ejemplo: Un único dato de entrada
x = np.array([0.5, -0.5])  # Dimensión: 2
y_true = 1.0

# Paso forward
# Capa oculta
z1 = np.dot(W1, x) + b1         # z1: vector de 2
a1 = phi(z1, beta)              # a1: vector de 2, activación personalizada

# Capa de salida (activación lineal)
z2 = np.dot(W2, a1) + b2        # z2: escalar
y_pred = z2

# Función de pérdida (error cuadrático)
loss = 0.5 * (y_pred - y_true)**2

print("Paso forward:")
print("x =", x)
print("z1 =", z1)
print("a1 =", a1)
print("z2 =", z2)
print("y_pred =", y_pred)
print("Perdida =", loss)

# Paso backward
# Derivada de la pérdida respecto a z2
dL_dz2 = y_pred - y_true   # Escalar

# Gradientes para la capa de salida
dL_dW2 = dL_dz2 * a1       # (1x2)
dL_db2 = dL_dz2          # Escalar

# Propagamos el gradiente a la capa oculta
# Nota: W2 tiene forma (1,2) -> se transpone para distribuir el error a cada neurona oculta
delta1 = (W2.T * dL_dz2).flatten()  # vector de 2
# Multiplicamos por la derivada de la activación
delta1 = delta1 * dphi_dz(z1, beta)  # elemento a elemento

# Gradientes para la capa oculta
dL_dW1 = np.outer(delta1, x)   # (2x2)
dL_db1 = delta1               # (2,)

# Gradiente respecto a beta (se suma la contribución de cada neurona oculta)
dL_dbeta = np.sum(delta1 * dphi_dbeta(z1))

print("\nPaso backward:")
print("dL_dz2 =", dL_dz2)
print("dL_dW2 =", dL_dW2)
print("dL_db2 =", dL_db2)
print("delta1 =", delta1)
print("dL_dW1 =", dL_dW1)
print("dL_db1 =", dL_db1)
print("dL_dbeta =", dL_dbeta)


#### Discusión sobre explosión o desvanecimiento del gradiente y estrategia de inicialización

La función de activación propuesta es:
$$
\phi(z) = z\,\sigma(z) + \beta\,\sin(z).
$$
Observaciones:

1. **Comportamiento de la parte $z\,\sigma(z)$:**  
   Esta parte es similar a la función *swish*, la cual ha demostrado tener buenas propiedades para evitar el desvanecimiento del gradiente, pues su derivada se mantiene en valores razonables para un amplio rango de $z$.

2. **Impacto del término $\beta\,\sin(z)$:**  
   - **Si $\beta$ es muy grande:**  
     La derivada de este término es $\beta\,\cos(z)$. Para ciertos valores de $z$ (por ejemplo, cuando $\cos(z)$ está cercano a 1), un valor elevado de $\beta$ puede amplificar excesivamente el gradiente, contribuyendo a la **explosión del gradiente**.
   - **Si $\beta$ es muy pequeño (o cero):**  
     La contribución del término oscilatorio se vuelve insignificante. En ese caso, el comportamiento se asemeja al de *swish*, lo que es deseable en muchos contextos, pero se pierde la potencial ventaja de la corrección oscilatoria.

3. **Efecto combinado:**  
   La presencia del término oscilatorio (debido a $\sin(z)$) puede introducir fluctuaciones en la magnitud del gradiente. Esto significa que, dependiendo de la región en la que se encuentre $z$, se puede observar tanto un aumento como una disminución brusca del gradiente.

##### **Estrategia de inicialización de $\beta$**:

Para evitar problemas de explosión del gradiente, se recomienda **inicializar $\beta$ con un valor pequeño**. Algunas estrategias posibles son:
- **Inicialización cercana a cero:**  
  Por ejemplo, usar $\beta = 0.0$ o un valor pequeño positivo como $0.01$ o $0.1$. Esto asegura que al comienzo la contribución oscilatoria no domine la dinámica del gradiente.
- **Inicialización adaptativa:**  
  Se puede considerar entrenar $\beta$ con una tasa de aprendizaje separada o regularización adicional que controle su crecimiento durante el entrenamiento.

Iniciar $\beta$ con un valor pequeño (por ejemplo, $\beta=0.1$) es una estrategia prudente para que la función de activación comience con un comportamiento similar al de *swish* y, a medida que el entrenamiento progrese, el modelo aprenda si la corrección oscilatoria aporta mejoras sin generar inestabilidad en la propagación del gradiente.


### **Pregunta 3**

####  Función de pérdida total

Supongamos que nuestro modelo tiene tres objetivos que se desean optimizar conjuntamente:

1. **Cross Entropy (CE)** para clasificación:  
   Sea la salida del modelo para clasificación $\hat{\mathbf{y}}$ y la etiqueta verdadera $\mathbf{y}$ (usando one-hot encoding, por ejemplo). La pérdida se define como:
   $$
   L_{\text{CE}} = - \sum_{i} y_i \log (\hat{y}_i).
   $$

2. **Kullback-Leibler Divergence (KL)** para alinear distribuciones de salidas:  
   Sea $q$ la distribución "objetivo" y $p$ la distribución generada por el modelo. La divergencia se define como:
   $$
   D_{\text{KL}}(q \parallel p) = \sum_{i} q_i \log \frac{q_i}{p_i}.
   $$
   (En aplicaciones como VAEs, se usa para forzar la distribución latente a acercarse a una distribución prior).

3. **Reconstruction Loss (MSE)** para forzar a una capa interna a autoencodificar parte de la representación:  
   Sea la representación original $\mathbf{x}$ y la reconstruida $\hat{\mathbf{x}}$. El MSE se define como:
   $$
   L_{\text{MSE}} = \frac{1}{N} \sum_{j} (x_j - \hat{x}_j)^2.
   $$

Cada uno de estos términos se pondera con un hiperparámetro distinto. Así, la función de pérdida total es:
$$
\mathcal{L}_{\text{total}} = \lambda_{\text{CE}} \, L_{\text{CE}} + \lambda_{\text{KL}} \, D_{\text{KL}} + \lambda_{\text{MSE}} \, L_{\text{MSE}},
$$
donde $\lambda_{\text{CE}}$, $\lambda_{\text{KL}}$ y $\lambda_{\text{MSE}}$ son hiperparámetros que determinan la contribución de cada término.

#### Retropropagación de los gradientes

El entrenamiento se realiza usando retropropagación. El flujo es el siguiente:

1. **Cálculo de cada término de pérdida:**  
   - En la rama de clasificación se computa $L_{\text{CE}}$.
   - En la rama de alineación de distribuciones se calcula $D_{\text{KL}}$.
   - En la rama de reconstrucción se evalúa $L_{\text{MSE}}$.

2. **Suma ponderada y retropropagación:**  
   Se suma la pérdida total y, usando la regla de la cadena, se retropropaga el gradiente desde la salida del modelo hasta los parámetros internos.  
   Es decir, para cada parámetro $\theta$ se tiene:
   $$
   \frac{\partial \mathcal{L}_{\text{total}}}{\partial \theta} = \lambda_{\text{CE}} \, \frac{\partial L_{\text{CE}}}{\partial \theta} + \lambda_{\text{KL}} \, \frac{\partial D_{\text{KL}}}{\partial \theta} + \lambda_{\text{MSE}} \, \frac{\partial L_{\text{MSE}}}{\partial \theta}.
   $$

3. **Orden y posibles incompatibilidades:**  
   - **Orden:**  
     La retropropagación se realiza "en paralelo" en el sentido de que, en la última capa, se suman los gradientes provenientes de cada término. Si las ramas comparten parámetros (por ejemplo, en capas intermedias), cada gradiente parcial se suma y se utiliza para actualizar los parámetros.
   - **Riesgo de incompatibilidad:**  
     Los tres términos pueden tener escalas y comportamientos muy distintos. Por ejemplo, si el término KL tiene una magnitud mucho mayor que los otros, puede "desviar" el descenso de gradiente de manera que la red priorice la alineación de distribuciones en detrimento de la clasificación o la reconstrucción. Por ello es crucial ajustar los coeficientes $\lambda$ de modo que ninguno domine excesivamente la actualización de los parámetros compartidos.

#### Ejemplo numérico

Consideremos un ejemplo simplificado en el que la variable de optimización es un único parámetro $x$.  
Imaginemos que, usando solo CE y MSE, el "mínimo ideal" se alcanza en $x=2$. Esto lo representamos con una función de pérdida de la forma:
$$
L_{\text{CE}} + L_{\text{MSE}} = (x-2)^2 + (x-2)^2 = 2\,(x-2)^2.
$$

Ahora, añadamos un término de KL que “empuje” $x$ hacia $3$:
$$
D_{\text{KL}} = (x-3)^2.
$$

La pérdida total (con $\lambda_{\text{CE}}=\lambda_{\text{MSE}}=1$ y $\lambda_{\text{KL}}=1$) es:
$$
L_{\text{total}}(x) = 2\,(x-2)^2 + (x-3)^2.
$$

##### **Cálculo analítico del mínimo**

Derivamos $L_{\text{total}}(x)$ respecto a $x$:
$$
\frac{dL_{\text{total}}}{dx} = 2 \cdot 2 (x-2) + 2 (x-3) = 4(x-2) + 2(x-3).
$$
Igualamos a cero:
$$
4(x-2) + 2(x-3) = 0 \quad \Rightarrow \quad 4x - 8 + 2x - 6 = 0 \quad \Rightarrow \quad 6x - 14 = 0,
$$
de donde:
$$
x = \frac{14}{6} \approx 2.33.
$$
Esto muestra que, al incluir el término KL, el óptimo se "desvía" de $x=2$ a $x \approx 2.33$.


A continuación se presenta un código en Python que ilustra este comportamiento:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Coeficientes de ponderación
lambda_ce = 1.0
lambda_mse = 1.0
lambda_kl = 1.0

def loss_total(x, lambda_ce, lambda_kl, lambda_mse):
    # Supongamos que CE y MSE "idealmente" minimizan en x=2 y KL en x=3
    loss_ce = (x - 2)**2
    loss_mse = (x - 2)**2
    loss_kl = (x - 3)**2
    return lambda_ce * loss_ce + lambda_kl * loss_kl + lambda_mse * loss_mse

# Generar valores de x para visualizar la función de pérdida
xs = np.linspace(0, 5, 100)
ys = loss_total(xs, lambda_ce, lambda_kl, lambda_mse)

# Identificar el mínimo
min_x = xs[np.argmin(ys)]
min_y = np.min(ys)

# Visualización
plt.figure(figsize=(8, 5))
plt.plot(xs, ys, label="Pérdida total")
plt.axvline(2, color='r', linestyle='--', label="Mínimo con CE+MSE (x=2)")
plt.axvline(min_x, color='g', linestyle='--', label=f"Mínimo Total (x={min_x:.2f})")
plt.xlabel("x")
plt.ylabel("Pérdida")
plt.title("Ejemplo numérico: influencia del término KL")
plt.legend()
plt.show()

print("El mínimo de la pérdida total se alcanza en x =", min_x)


#### Explicación de los resultados

- **Sin el término KL:**  
  Si solo se consideran $L_{\text{CE}}$ y $L_{\text{MSE}}$, la función de pérdida es $2\,(x-2)^2$ y el mínimo se alcanza en $x=2$.

- **Con el término KL:**  
  Al agregar $(x-3)^2$ (ponderado por $\lambda_{KL}=1$), el mínimo de la pérdida total se desplaza hacia $x \approx 2.33$. Esto ilustra que la inclusión de la divergencia KL “desvía” el óptimo de la solución que se obtendría solo con CE y MSE.

En un modelo real, esta desviación puede ser beneficiosa o perjudicial según el objetivo de alinear distribuciones. Sin embargo, si no se pondera adecuadamente, el término KL puede dominar y forzar a la red a aprender representaciones que se adapten a la distribución prior en detrimento de la precisión en clasificación o reconstrucción.

### **Pregunta 4**

#### Descripción del procedimiento

1. **Generar dataset 2D:**  
   Se crea un conjunto de datos bidimensional en el que la etiqueta se define de forma sencilla. Por ejemplo, se puede asignar la etiqueta  
   $$
   y = \begin{cases} 1 & \text{si } x + y \ge 0 \\ 0 & \text{si } x + y < 0 \end{cases}
   $$
   para cada punto $(x,y)$.

2. **Entrenar un perceptrón multicapa:**  
   Se construye una red con, por ejemplo, dos capas ocultas (usando 10 neuronas cada una) y una salida con activación sigmoidea (para clasificación binaria). La arquitectura es:
   - **Entrada:** 2 unidades  
   - **Capa oculta 1:** 10 neuronas con función de activación *tanh*  
   - **Capa oculta 2:** 10 neuronas con función de activación *tanh*  
   - **Salida:** 1 neurona con activación sigmoidea  
   Se usa la *cross entropy* como función de pérdida.

3. **Visualizar la región de decisión:**  
   Se genera una malla de puntos sobre el plano. Para cada punto se calcula la salida del modelo y se traza la frontera (por ejemplo, usando un diagrama de contorno) de la región en la que el modelo predice la clase 1 versus 0.

4. **Cálculo de gradientes y mapa de calor:**  
   Para cada punto de la malla se realiza un forward pass y se computa la pérdida (suponiendo la etiqueta "verdadera" determinada por la función de generación, es decir, 1 si $x+y\ge0$ y 0 en otro caso). Luego se hace un backward pass *manualmente* para obtener el gradiente de la pérdida con respecto a los pesos de la primera capa. Finalmente se calcula la magnitud (por ejemplo, la norma L2) de este gradiente y se representa en un mapa de calor superpuesto al diagrama del plano.

A continuación se muestra el código que implementa este procedimiento.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Generar dataset 2D
def generate_dataset(n_samples=200, lim=3.0, random_seed=42):
    np.random.seed(random_seed)
    # Generamos puntos uniformemente en [-lim, lim]
    X = np.random.uniform(-lim, lim, (n_samples, 2))
    # Etiqueta: 1 si x+y>=0, 0 de lo contrario
    y = (X[:, 0] + X[:, 1] >= 0).astype(np.float32)
    return X, y

X_train, y_train = generate_dataset(n_samples=300)

# Visualizamos el dataset
plt.figure(figsize=(6,5))
plt.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], color='red', label="Clase 0")
plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], color='blue', label="Clase 1")
plt.legend()
plt.title("Conjunto de datos 2D")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# 2. Definir y entrenar la red neuronal
# Arquitectura: 2 -> 10 -> 10 -> 1
# Usaremos activación tanh en las capas ocultas y sigmoid en la salida.
# Funciones de activación y sus derivadas
def tanh(z):
    return np.tanh(z)

def dtanh(z):
    return 1 - np.tanh(z)**2

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def dsigmoid(z):
    s = sigmoid(z)
    return s * (1-s)

# Inicialización de pesos (usamos inicialización sencilla tipo Xavier para tanh)
def init_weights(n_in, n_out):
    bound = np.sqrt(6 / (n_in + n_out))
    return np.random.uniform(-bound, bound, (n_out, n_in))

# Dimensiones
n_input = 2
n_hidden1 = 10
n_hidden2 = 10
n_output = 1

# Inicializamos pesos y sesgos
W1 = init_weights(n_input, n_hidden1)   # (10,2)
b1 = np.zeros((n_hidden1, 1))             # (10,1)
W2 = init_weights(n_hidden1, n_hidden2)   # (10,10)
b2 = np.zeros((n_hidden2, 1))             # (10,1)
W3 = init_weights(n_hidden2, n_output)    # (1,10)
b3 = np.zeros((n_output, 1))              # (1,1)

# Función forward para un solo ejemplo (x es vector columna de shape (2,1))
def forward(x):
    z1 = np.dot(W1, x) + b1       # (10,1)
    a1 = tanh(z1)
    z2 = np.dot(W2, a1) + b2      # (10,1)
    a2 = tanh(z2)
    z3 = np.dot(W3, a2) + b3      # (1,1)
    a3 = sigmoid(z3)
    cache = (x, z1, a1, z2, a2, z3, a3)
    return a3, cache

# Función de pérdida: Cross Entropy para clasificación binaria
def loss_fn(y_pred, y_true):
    # Para evitar log(0), sumamos una pequeña constante
    eps = 1e-8
    return - (y_true * np.log(y_pred+eps) + (1 - y_true)*np.log(1-y_pred+eps))

# Entrenamiento sencillo con gradiente descendente (por lotes)
learning_rate = 0.01
epochs = 200

# Convertimos X_train a shape (2, n_samples) y y_train a (1, n_samples)
X_train_T = X_train.T  # (2, n_samples)
y_train_T = y_train.reshape(1, -1)  # (1, n_samples)

for epoch in range(epochs):
    total_loss = 0
    # Gradientes acumulados (inicializados en cero)
    dW1 = np.zeros_like(W1)
    db1 = np.zeros_like(b1)
    dW2 = np.zeros_like(W2)
    db2 = np.zeros_like(b2)
    dW3 = np.zeros_like(W3)
    db3 = np.zeros_like(b3)
    
    n_samples = X_train_T.shape[1]
    for i in range(n_samples):
        # Extraer el ejemplo i-esimo
        x = X_train_T[:, i].reshape(-1,1)   # (2,1)
        y_true = y_train_T[:, i].reshape(1,1) # (1,1)
        
        # Forward pass
        a3, cache = forward(x)
        total_loss += loss_fn(a3, y_true)
        
        # Backward pass (derivadas manuales)
        x, z1, a1, z2, a2, z3, a3 = cache
        
        # Gradiente de la pérdida con respecto a z3
        dz3 = a3 - y_true  # (1,1)  ; derivada de cross entropy con sigmoid
        dW3 += np.dot(dz3, a2.T)   # (1,10)
        db3 += dz3
        
        # Propagar a la capa 2
        da2 = np.dot(W3.T, dz3)    # (10,1)
        dz2 = da2 * dtanh(z2)      # (10,1)
        dW2 += np.dot(dz2, a1.T)   # (10,10)
        db2 += dz2
        
        # Propagar a la capa 1 (pesos principales)
        da1 = np.dot(W2.T, dz2)    # (10,1)
        dz1 = da1 * dtanh(z1)      # (10,1)
        dW1 += np.dot(dz1, x.T)    # (10,2)
        db1 += dz1
        
    # Actualización de parámetros (dividiendo por el número de muestras)
    W1 -= learning_rate * dW1 / n_samples
    b1 -= learning_rate * db1 / n_samples
    W2 -= learning_rate * dW2 / n_samples
    b2 -= learning_rate * db2 / n_samples
    W3 -= learning_rate * dW3 / n_samples
    b3 -= learning_rate * db3 / n_samples
    
    if epoch % 20 == 0:
        print(f"Epoca {epoch}, Loss: {(total_loss/n_samples).item():.4f}")

# Se asume que la red ya está entrenada.
# 3. Visualización de la región de decisión
# Generamos una malla de puntos en el plano
xx, yy = np.meshgrid(np.linspace(-3,3,200), np.linspace(-3,3,200))
grid_points = np.c_[xx.ravel(), yy.ravel()]  # (n_points, 2)

# Función de predicción
def predict(x):
    a3, _ = forward(x.reshape(-1,1))
    return a3[0,0]

Z = np.array([predict(pt) for pt in grid_points])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8,6))
# Diagrama de contorno de la región de decisión
plt.contourf(xx, yy, Z, levels=np.linspace(0,1,20), cmap='coolwarm', alpha=0.6)
plt.colorbar(label="Probabilidad predicha")
# Superponer el dataset de entrenamiento
plt.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], color='red', edgecolor='k', label="Clase 0")
plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], color='blue', edgecolor='k', label="Clase 1")
plt.legend()
plt.title("Región de decisión del MLP")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# 4. Cálculo del gradiente (magnitud) respecto a W1 en cada punto de la malla
def grad_W1_norm(x, y):
    """
    Para un punto (x,y) se computa la magnitud (norma L2) del gradiente de la pérdida
    (cross entropy) respecto a los pesos de la primera capa (W1).
    Se asume la etiqueta verdadera según la regla: y_true=1 si x+y>=0, 0 si no.
    """
    x_vec = np.array([x, y]).reshape(-1,1)  # (2,1)
    # Etiqueta verdadera (según la regla de generación)
    y_true = np.array([[1.0]]) if (x+y >= 0) else np.array([[0.0]])
    # Forward pass
    a3, cache = forward(x_vec)
    loss = loss_fn(a3, y_true)
    
    # Backward pass para obtener gradiente de W1 (igual que en el entrenamiento, pero para un único ejemplo)
    x_in, z1, a1, z2, a2, z3, a3 = cache
    dz3 = a3 - y_true  # (1,1)
    # Propagación hacia la capa 2
    da2 = np.dot(W3.T, dz3)    # (10,1)
    dz2 = da2 * dtanh(z2)      # (10,1)
    # Propagación hacia la capa 1 (pesos principales)
    da1 = np.dot(W2.T, dz2)    # (10,1)
    dz1 = da1 * dtanh(z1)      # (10,1)
    # Gradiente de W1 para este ejemplo: dW1 = dz1 * x^T, de forma (10,2)
    dW1_local = np.dot(dz1, x_in.T)
    # Retornamos la norma L2 de este gradiente
    return np.linalg.norm(dW1_local)

# Computamos la magnitud del gradiente para cada punto de la malla
grad_norms = np.array([grad_W1_norm(pt[0], pt[1]) for pt in grid_points])
grad_norms = grad_norms.reshape(xx.shape)

# Mapa de calor del gradiente superpuesto a la región de decisión
plt.figure(figsize=(8,6))
plt.contourf(xx, yy, Z, levels=np.linspace(0,1,20), cmap='coolwarm', alpha=0.6)
heatmap = plt.contourf(xx, yy, grad_norms, levels=50, cmap='viridis', alpha=0.5)
plt.colorbar(heatmap, label="Norma del gradiente (W1)")
plt.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], color='red', edgecolor='k', label="Clase 0")
plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], color='blue', edgecolor='k', label="Clase 1")
plt.legend()
plt.title("Mapa de calor de la norma del gradiente respecto a W1")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# 5. Análisis

print("""
Análisis:
- En zonas donde la red está muy segura (por ejemplo, donde la probabilidad predicha se acerca a 0 o 1) 
  el gradiente de la cross entropy tiende a ser pequeño, lo que indica saturación de la función de activación 
  (en particular, tanh o sigmoide saturan para valores extremos).
- En las regiones cercanas a la frontera de decisión, la salida varía más rápidamente y la pérdida es mayor, 
  generando gradientes de mayor magnitud.
""")


#### Discusión sobre la inicialización de pesos y la elección de la función de activación

- **Inicialización de pesos:**  
  - Una **inicialización adecuada** (por ejemplo, Xavier o He) asegura que los valores de activación en las capas intermedias no se dispersen en zonas de saturación.  
  - Si los pesos se inicializan con valores muy grandes, las activaciones (por ejemplo, tanh o sigmoide) tienden a saturarse (valores muy cercanos a 1 o -1, o 0 y 1 en el caso de sigmoide), y la derivada (y por ende el gradiente) se vuelve muy pequeña. Esto se reflejará en el mapa de calor como zonas de gradiente casi nulo.  
  - Una inicialización inadecuada (por ejemplo, con valores demasiado pequeños) puede provocar que la señal se debilite desde la entrada, lo que también afecta el aprendizaje.

- **Elección de la función de activación:**  
  - **Funciones saturantes (sigmoide, tanh):**  
    Estas funciones tienden a “aplanarse” para entradas muy positivas o muy negativas. Esto causa que en estas regiones la derivada sea muy pequeña y, en consecuencia, se produzca el *desvanecimiento del gradiente*. En el mapa de calor se verán áreas con gradientes bajos.  
  - **Funciones no saturantes (ReLU, Leaky ReLU):**  
    Estas funciones, en particular para valores positivos, no saturan (o lo hacen de forma moderada). Esto favorece que el gradiente se mantenga en niveles adecuados en gran parte del dominio, lo que se reflejará en un mapa de calor con regiones de gradiente más “uniforme” (aunque pueden aparecer zonas muertas en ReLU si muchos neuronas quedan en 0).

Tanto la correcta inicialización de los pesos como la elección de la función de activación son factores cruciales que determinan en qué zonas del espacio de entrada el modelo aprende de forma efectiva (con gradientes significativos) y en cuáles se produce saturación (con gradientes muy pequeños). Ajustar estos aspectos ayuda a evitar problemas de desvanecimiento o explosión de gradientes y a lograr una convergencia más estable durante el entrenamiento.

### **Pregunta 5**

#### Escenario propuesto

Imaginemos un escenario en el que cada muestra del dataset tiene dos componentes:
- **Texto:** Una secuencia de oraciones cortas. Cada oración se tokeniza y se proyecta a un espacio de embeddings, de dimensión $d_{\text{model}}$.
- **Imagen:** Una imagen asociada que se procesa (por ejemplo, mediante una CNN o un extractor de características) para obtener descriptores o *features*. Supongamos que se extraen $M$ vectores (por ejemplo, regiones o patches) de dimensión $d_{\text{img}}$.

Para fusionar ambos modos, se puede utilizar un **mecanismo de auto-atención** en cada rama (texto e imagen) y luego aplicar un **mecanismo de cross-attention** para que una modalidad aprenda a "atender" a la otra.

#### Self-Attention para cada modalidad

##### **Para el texto**

Sea $X_{\text{text}} \in \mathbb{R}^{T \times d_{\text{model}}}$ la matriz de embeddings de una secuencia de $T$ tokens. Se proyecta a un espacio compartido (de dimensión $d_k$) para calcular queries, keys y values:
$$
\begin{aligned}
Q_{\text{text}} &= X_{\text{text}} W_{\text{text}}^Q \quad &&\in \mathbb{R}^{T \times d_k} \\
K_{\text{text}} &= X_{\text{text}} W_{\text{text}}^K \quad &&\in \mathbb{R}^{T \times d_k} \\
V_{\text{text}} &= X_{\text{text}} W_{\text{text}}^V \quad &&\in \mathbb{R}^{T \times d_v}
\end{aligned}
$$
La atención se calcula como:
$$
\text{Attention}(X_{\text{text}}) = \text{softmax}\left(\frac{Q_{\text{text}}K_{\text{text}}^\top}{\sqrt{d_k}}\right) V_{\text{text}}
$$

##### **Para la imagen**

Sea $X_{\text{img}} \in \mathbb{R}^{M \times d_{\text{img}}}$ la matriz de características extraídas de $M$ regiones o patches. De forma similar, se proyecta:
$$
\begin{aligned}
Q_{\text{img}} &= X_{\text{img}} W_{\text{img}}^Q \quad &&\in \mathbb{R}^{M \times d_k} \\
K_{\text{img}} &= X_{\text{img}} W_{\text{img}}^K \quad &&\in \mathbb{R}^{M \times d_k} \\
V_{\text{img}} &= X_{\text{img}} W_{\text{img}}^V \quad &&\in \mathbb{R}^{M \times d_v}
\end{aligned}
$$
Y se calcula la auto-atención:
$$
\text{Attention}(X_{\text{img}}) = \text{softmax}\left(\frac{Q_{\text{img}}K_{\text{img}}^\top}{\sqrt{d_k}}\right) V_{\text{img}}
$$

> **Nota sobre las dimensiones:**  
> Para que la multiplicación $Q K^\top$ sea válida, ambas matrices deben compartir la dimensión $d_k$. Por ello, en ambos casos (texto e imagen) se proyecta a un espacio de dimensión $d_k$. Esto es clave para el mecanismo de cross-attention, pues se requiere que queries y keys provenientes de distintas modalidades sean compatibles.

#### Cross-attention para la interacción entre modalidades

El cross-attention permite que una modalidad (por ejemplo, el texto) "atenga" a la otra (la imagen) para obtener una representación enriquecida.

Utilizando los queries del texto y los keys y values de la imagen, la atención cruzada se define como:
$$
\text{Attention}(X_{\text{text}}, X_{\text{img}}) = \text{softmax}\left(\frac{Q_{\text{text}}K_{\text{img}}^\top}{\sqrt{d_k}}\right) V_{\text{img}}
$$


De forma análoga, para que la imagen se beneficie de la información textual:
$$
\text{Attention}(X_{\text{img}}, X_{\text{text}}) = \text{softmax}\left(\frac{Q_{\text{img}}K_{\text{text}}^\top}{\sqrt{d_k}}\right) V_{\text{text}}
$$

> **Dimensiones involucradas:**  
> - $X_{\text{text}}$ tiene forma $(T, d_{\text{model}})$ y se proyecta a $(T, d_k)$.  
> - $X_{\text{img}}$ tiene forma $(M, d_{\text{img}})$ y se proyecta a $(M, d_k)$.  
>  
> Para que la operación de dot product entre $Q_{\text{text}}$ y $K_{\text{img}}^\top$ sea posible, ambas deben tener $d_k$ como dimensión final. De allí que se asuma un $d_k$ compartido entre las dos modalidades (o, en su defecto, se transforman para alcanzar dicha compatibilidad).


A continuación se muestra un ejemplo de cómo implementar estos mecanismos en PyTorch:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiModalAttention(nn.Module):
    def __init__(self, d_text, d_img, d_k, d_v):
        super(MultiModalAttention, self).__init__()
        # Proyecciones para el texto
        self.Wq_text = nn.Linear(d_text, d_k)
        self.Wk_text = nn.Linear(d_text, d_k)
        self.Wv_text = nn.Linear(d_text, d_v)
        
        # Proyecciones para la imagen
        self.Wq_img = nn.Linear(d_img, d_k)
        self.Wk_img = nn.Linear(d_img, d_k)
        self.Wv_img = nn.Linear(d_img, d_v)
    
    def self_attention(self, Q, K, V):
        # Calcula la atención escalada
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(Q.size(-1), dtype=torch.float32))
        attn = F.softmax(scores, dim=-1)
        return torch.matmul(attn, V)
    
    def self_attention_text(self, x_text):
        Q = self.Wq_text(x_text)
        K = self.Wk_text(x_text)
        V = self.Wv_text(x_text)
        return self.self_attention(Q, K, V)
    
    def self_attention_img(self, x_img):
        Q = self.Wq_img(x_img)
        K = self.Wk_img(x_img)
        V = self.Wv_img(x_img)
        return self.self_attention(Q, K, V)
    
    def cross_attention(self, Q, K, V):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(Q.size(-1), dtype=torch.float32))
        attn = F.softmax(scores, dim=-1)
        return torch.matmul(attn, V)
    
    def cross_attention_text_to_img(self, x_text, x_img):
        Q_text = self.Wq_text(x_text)
        K_img = self.Wk_img(x_img)
        V_img = self.Wv_img(x_img)
        return self.cross_attention(Q_text, K_img, V_img)
    
    def cross_attention_img_to_text(self, x_img, x_text):
        Q_img = self.Wq_img(x_img)
        K_text = self.Wk_text(x_text)
        V_text = self.Wv_text(x_text)
        return self.cross_attention(Q_img, K_text, V_text)

# Ejemplo de uso
# Supongamos que:
# - Las secuencias de texto tienen 10 tokens, cada uno con dimensión 512.
# - Las imágenes tienen 20 patches, cada uno con dimensión 1024.
# - Queremos proyectar a un espacio de dimensión d_k=64 y d_v=64.
batch_size = 2
T, d_text = 10, 512
M, d_img = 20, 1024
d_k, d_v = 64, 64
    
x_text = torch.randn(batch_size, T, d_text)
x_img = torch.randn(batch_size, M, d_img)
    
modelo = MultiModalAttention(d_text, d_img, d_k, d_v)
    
# Auto-atención para cada modalidad
out_text = modelo.self_attention_text(x_text)
out_img = modelo.self_attention_img(x_img)
    
# Cross-attention: texto atendiendo a imagen
cross_text_to_img = modelo.cross_attention_text_to_img(x_text, x_img)
    
# Cross-attention: imagen atendiendo a texto
cross_img_to_text = modelo.cross_attention_img_to_text(x_img, x_text)
    
print("Salida auto-atención texto:", out_text.shape)
print("Salida auto-atención imagen:", out_img.shape)
print("Salida cross-attention texto->imagen:", cross_text_to_img.shape)
print("Salida cross-attention imagen->texto:", cross_img_to_text.shape)


Este código demuestra cómo se pueden calcular las atenciones propias (self-attention) para cada modalidad y cómo implementar la atención cruzada (cross-attention).

#### **Ajuste del factor $\sqrt{d_k}$**

En la ecuación de atención escalada se utiliza el factor $\sqrt{d_k}$ para evitar que los productos escalares (dot products) crezcan demasiado al aumentar la dimensión, lo que puede llevar a que el softmax se sature y la distribución de probabilidades se vuelva muy "puntiaguda". 
Sin embargo, existen casos en los que podría justificarse ajustar este factor de forma distinta para cada modalidad:

1. **Diferentes distribuciones de características:**  
   Si las características del texto y de la imagen tienen distribuciones distintas (por ejemplo, varianzas diferentes) tras la proyección, el rango de los valores resultantes de los dot products también puede diferir.  
   - **Circunstancia:** Cuando los embeddings de imagen presentan mayor varianza (o están en una escala distinta) en comparación con los embeddings de texto.  
   - **Justificación:** Ajustar el factor $\sqrt{d_k}$ (o incluso aprender un factor de escala) para cada modalidad ayuda a normalizar las magnitudes y evita que una modalidad domine la otra en la atención cruzada.

2. **Dimensiones proyectadas diferentes:**  
   Si, por alguna razón, se decide proyectar cada modalidad a dimensiones $d_k^{\text{text}}$ y $d_k^{\text{img}}$ diferentes, se debería usar $\sqrt{d_k^{\text{text}}}$ o $\sqrt{d_k^{\text{img}}}$ según corresponda.  
   - **Circunstancia:** En modelos multimodales donde se cree que cada modalidad requiere una representación con distinta capacidad, por ejemplo, si se considera que la imagen tiene información más compleja y se usa un $d_k$ mayor.
   - **Justificación:** Esto asegura que la escala de los dot products se normalice adecuadamente en cada caso.

3. **Ajuste fino y aprendizaje del escalado:**  
   En algunos modelos, se pueden introducir parámetros aprendibles que modulan este factor de escala para cada modalidad o incluso para cada cabecera en la atención multi-cabecera.  
   - **Circunstancia:** Cuando se dispone de datos suficientes para aprender la escala óptima que permita equilibrar la contribución de cada modalidad.
   - **Justificación:** Un factor de escala ajustado (o aprendido) puede mejorar la convergencia y el rendimiento del modelo al equilibrar adecuadamente las diferencias intrínsecas entre los modos.

En resumen, aunque el factor $\sqrt{d_k}$ se introduce originalmente para normalizar los dot products en función de la dimensión, en escenarios multimodales donde las características de texto e imagen pueden tener escalas o distribuciones diferentes (o incluso dimensiones proyectadas distintas), ajustar (o aprender) este factor de forma separada puede ayudar a obtener una atención más equilibrada y estable.


### **Pregunta 6**

#### Diseño de la arquitectura

En una red recurrente “estándar” tenemos una dependencia de la forma:  
$$
h_t = f(x_t, h_{t-1})
$$
donde $h_t$ es el estado oculto en el tiempo $t$ y $x_t$ la entrada en ese instante.

Para introducir una conexión **no estándar** que forme un ciclo en el grafo computacional, podemos agregar un **módulo iterativo interno** en cada paso temporal que refina el estado oculto. Es decir, luego de calcular el estado inicial $h_t^0$ con la recurrencia clásica, se ejecuta una iteración interna que retroalimenta la misma salida hasta converger o hasta un número fijo de iteraciones. Así, la actualización se puede escribir como:

1. **Paso recurrente estándar:**  
   $$
   h_t^0 = \tanh(W_x x_t + W_h h_{t-1})
   $$
2. **Ciclo interno (iterativo):**  
   $$
   h_t^{(i+1)} = \tanh\Big(h_t^{(i)} + W_c\, h_t^{(i)}\Big) \quad \text{para } i=0,1,\ldots, N-1
   $$
   Finalmente, se toma:
   $$
   h_t = h_t^{(N)}
   $$

Esta conexión adicional $W_c\, h_t^{(i)}$ dentro de un bucle introduce un **ciclo** en el grafo computacional porque la misma variable $h_t$ se actualiza reiteradamente de forma recursiva durante el mismo paso temporal.

##### **Diagrama de bloques**

A continuación se muestra un diagrama esquemático en el que se ilustra la arquitectura:

```
              x_t
               │
               ▼
       +-----------------+
       |  Capa de Input  |
       +-----------------+
               │
               ▼
       +-----------------+
       | RNN Cell Standard| <-- h_{t-1}
       +-----------------+
               │
               ▼
            h_t^0
               │
         ┌─────┴─────┐
         │  Módulo   │  ←─────+
         │  Ciclo    │        │
         │ Iterativo │        │
         └─────┬─────┘        │
               │              │
          (Iteración interna) │
               │              │
               +──────────────+
               │
               ▼
             h_t  (Salida final)
```

> **Nota:**  
> El módulo ciclo iterativo puede ejecutarse un número fijo de veces (por ejemplo, $N=3$) o hasta que se cumpla una condición de convergencia. Esta conexión interna no lineal introduce un ciclo en el grafo que debe resolverse de forma iterativa.

##### **Diferencias entre frameworks de grafo estático y dinámico**

- **Grafos estáticos:**  
  Frameworks como TensorFlow 1.x requieren definir de antemano toda la estructura del grafo computacional. Las conexiones cíclicas o iteraciones cuyo número de pasos depende de la entrada deben "desenrollarse" de forma fija. Esto implica:
  - Un mayor esfuerzo en la definición manual del bucle.
  - La necesidad de utilizar estructuras como `tf.while_loop` con límites fijos o condiciones predefinidas.
  - La dificultad de manejar ciclos cuyo comportamiento dependa dinámicamente de la ejecución.

- **Grafos dinámicos (Eager Mode, PyTorch, JAX):**  
  En estos frameworks, el grafo se construye sobre la marcha durante la ejecución (modo "eager"). Esto permite:
  - Programar ciclos o iteraciones internas de forma natural utilizando estructuras de control de Python (como `for` o `while`).
  - Ajustar de forma dinámica el número de iteraciones en función de la condición de convergencia u otros criterios.
  - Mayor flexibilidad para modelar dependencias cíclicas en el grafo computacional sin tener que "desenrollar" previamente el ciclo.


El siguiente código implementa una celda recurrente que, en cada paso temporal, realiza una iteración interna (ciclo) para refinar el estado oculto:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CycleRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size, cycle_steps=3):
        """
        Parámetros:
          - input_size: Dimensión de la entrada x_t.
          - hidden_size: Dimensión del estado oculto h_t.
          - cycle_steps: Número de iteraciones internas para el ciclo.
        """
        super(CycleRNNCell, self).__init__()
        self.hidden_size = hidden_size
        self.cycle_steps = cycle_steps
        
        # Recurrencia estándar: de x_t y h_{t-1} a h_t^0
        self.Wx = nn.Linear(input_size, hidden_size)
        self.Wh = nn.Linear(hidden_size, hidden_size)
        
        # Conexión adicional para el ciclo interno
        self.Wc = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x, h_prev):
        # Paso recurrente estándar
        h = torch.tanh(self.Wx(x) + self.Wh(h_prev))
        # Ciclo interno de refinamiento
        for _ in range(self.cycle_steps):
            h = torch.tanh(h + self.Wc(h))
        return h

class CycleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, cycle_steps=3):
        super(CycleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.cell = CycleRNNCell(input_size, hidden_size, cycle_steps)
    
    def forward(self, inputs):
        """
        inputs: Tensor de forma (seq_len, batch_size, input_size)
        """
        seq_len, batch_size, _ = inputs.size()
        # Inicializamos h0 en cero
        h = torch.zeros(batch_size, self.hidden_size, device=inputs.device)
        outputs = []
        for t in range(seq_len):
            x_t = inputs[t]
            h = self.cell(x_t, h)
            outputs.append(h.unsqueeze(0))
        outputs = torch.cat(outputs, dim=0)
        return outputs

# Ejemplo de uso:
seq_len = 5
batch_size = 2
input_size = 10
hidden_size = 20
cycle_steps = 3
    
# Crear una secuencia de entrada aleatoria
sample_input = torch.randn(seq_len, batch_size, input_size)
rnn_model = CycleRNN(input_size, hidden_size, cycle_steps)
output = rnn_model(sample_input)
print("Forma de la salida:", output.shape)


En este ejemplo se observa cómo se introduce un ciclo interno en la celda recurrente. Gracias al modo dinámico (eager mode) de PyTorch, el bucle `for` se ejecuta de forma natural durante la ejecución, sin necesidad de definir previamente un grafo estático.


#### Inconvenientes y estrategias de mitigación en ciclos dinámicos

##### **Inconvenientes o riesgos**

1. **Inestabilidad numérica y explosión/desvanecimiento de gradientes:**
   - **Riesgo:**  
     La presencia de ciclos internos iterativos puede provocar que los gradientes se acumulen de forma excesiva (explosión) o se atenúen demasiado (desvanecimiento) a través de las iteraciones, lo que afecta el proceso de entrenamiento.
   - **Estrategias de nitigación:**
     - **Recorte de gradientes (gradient clipping):** Limitar el valor máximo de los gradientes para evitar explosiones.
     - **Normalización de activaciones:** Usar técnicas como batch normalization o layer normalization dentro del ciclo para estabilizar las activaciones.
     - **Inicialización adecuada:** Escoger inicializaciones de parámetros que eviten la amplificación excesiva o la atenuación de la señal.

2. **Aumento en la complejidad computacional y en el uso de memoria:**
   - **Riesgo:**  
     Los ciclos dinámicos implican múltiples iteraciones internas en cada paso temporal, lo que incrementa el costo computacional y la memoria necesaria para almacenar las activaciones y gradientes durante la retropropagación.
   - **Estrategias de mitigación:**
     - **Limitar el número de iteraciones:** Establecer un número fijo y moderado de pasos iterativos en el ciclo para balancear refinamiento y costo computacional.
     - **Detección de convergencia:** Implementar un criterio de parada temprana basado en la convergencia de las activaciones, de modo que se realicen únicamente las iteraciones necesarias.
     - **Truncamiento de backpropagation through time (BPTT):** En contextos recurrentes, limitar la propagación de gradientes a un número fijo de pasos para evitar el almacenamiento excesivo de información.

### **Pregunta 7**

En *knowledge distillation* se transfiere el conocimiento de un modelo complejo y grande (*teacher*) a uno más pequeño (*student*). El procedimiento es el siguiente:

1. **Generación de distribuciones suaves:**  
   El *teacher* procesa una entrada y genera unos logits $ z_i^{(T)} $ para cada una de las 10 clases. Para convertir estos logits en una distribución de probabilidad "suave", se utiliza la función *softmax* con un parámetro de temperatura $\tau$. Esto se escribe como:
   $$
   p_i^{(T)} = \frac{\exp\left(z_i^{(T)}/\tau\right)}{\sum_{j=1}^{10} \exp\left(z_j^{(T)}/\tau\right)}.
   $$
   El efecto de $\tau > 1$ es suavizar la distribución, revelando relaciones entre clases que no se notarían con una temperatura de 1.

2. **Imitación por el student:**  
   El *student* genera sus propios logits $ z_i^{(S)} $ y se le aplica la misma función *softmax* con la misma temperatura:
   $$
   p_i^{(S)} = \frac{\exp\left(z_i^{(S)}/\tau\right)}{\sum_{j=1}^{10} \exp\left(z_j^{(S)}/\tau\right)}.
   $$

3. **Cálculo de la pérdida:**  
   El objetivo es que la salida del *student* imite la del *teacher*. Para ello se utiliza la *Cross Entropy* entre la distribución del *teacher* (soft labels) y la del *student*. La fórmula exacta de la pérdida es:
   $$
   L_{KD} = -\tau^2 \sum_{i=1}^{10} p_i^{(T)} \log\left(p_i^{(S)}\right).
   $$
   Aquí, el factor $\tau^2$ se incluye para compensar la escala de los gradientes cuando se utiliza una temperatura alta. Esta pérdida mide cuán cercana es la distribución predicha por el *student* a la distribución "suave" del *teacher*.


El siguiente fragmento de código ilustra el cálculo de la pérdida en un entorno de PyTorch:

In [ ]:
import torch
import torch.nn.functional as F

def knowledge_distillation_loss(student_logits, teacher_logits, temperature):
    """
    Calcula la pérdida de distillation usando la entropía cruzada entre las distribuciones suavizadas.
    
    Args:
        student_logits (Tensor): Logits del modelo student de tamaño (B, 10).
        teacher_logits (Tensor): Logits del modelo teacher de tamaño (B, 10).
        temperature (float): Parámetro de temperatura τ.
    
    Returns:
        Tensor: Valor de la pérdida.
    """
    # Obtener las distribuciones de probabilidad suaves
    teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
    student_log_probs = F.log_softmax(student_logits / temperature, dim=1)
    
    # Calcular la pérdida (KL Divergence equivale a la Cross Entropy en este caso)
    loss = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean') * (temperature ** 2)
    return loss


En este ejemplo se usa la divergencia KL, que es equivalente a minimizar la entropía cruzada cuando se proporcionan las distribuciones suaves del teacher.

#### Estimación de memoria en el paso forward y backward

Supongamos que el modelo *student* tiene $ M_s $ parámetros y se evalúa en *minibatches* de tamaño $ B $. Es importante distinguir entre la memoria ocupada por los parámetros y la memoria utilizada para almacenar las activaciones y gradientes durante el entrenamiento.

##### **El paso forward  (activaciones):**

- **Parámetros:**  
  El modelo *student* tiene $ M_s $ parámetros, y cada uno (suponiendo 32 bits, o 4 bytes) ocupará:
  $$
  \text{Memoria de parámetros} \approx M_s \times 4 \quad \text{(bytes)}.
  $$

- **Activaciones:**  
  Durante el forward pass, se deben almacenar las activaciones intermedias de cada capa para cada muestra del minibatch. Si bien el tamaño exacto depende de la arquitectura, se puede asumir que la memoria para activaciones es proporcional al tamaño del minibatch y a la “complejidad” del modelo. De forma simplificada:
  $$
  \text{Memoria activaciones} \sim B \times \alpha \, M_s,
  $$
  donde $\alpha$ es un factor que depende del número de capas y la arquitectura (por ejemplo, si cada parámetro tiene una activación asociada o si se deben almacenar activaciones intermedias de mayor dimensión).

##### **El paso backward (gradientes):**

Durante el backward pass se deben almacenar:
- Los gradientes de cada parámetro (del mismo orden que $ M_s $).
- Los gradientes de las activaciones intermedias, lo que puede duplicar (o aumentar) el requerimiento de memoria respecto al forward.

En términos aproximados:
$$
\text{Memoria backward} \sim \text{Memoria activaciones} + \text{Memoria gradientes} \approx 2 \times (B \times \alpha \, M_s).
$$
Es decir, se puede estimar que el backward pass requiere aproximadamente el doble de la memoria que el forward pass, aunque este factor puede variar según la implementación y la arquitectura.

##### Comparación con el modelo *teacher*

Si el modelo *teacher* tiene $ M_t $ parámetros, y dado que $ M_t \gg M_s $:
- **Paso forward :**  
  La memoria para almacenar activaciones se escala aproximadamente como:
  $$
  \text{Memoria activaciones (teacher)} \sim B \times \alpha \, M_t,
  $$
  lo que es mucho mayor que para el *student*.
  
- **Paso backward:**  
  Similarmente, el requerimiento de memoria para almacenar gradientes se escala con $ M_t $. Por lo tanto:
  $$
  \text{Memoria backward (teacher)} \sim 2 \times (B \times \alpha \, M_t).
  $$

##### Relación entre reducción de memoria e inferencia

Reducir el número de parámetros de $ M_t $ a $ M_s $ implica:
- **Menor almacenamiento de activaciones y gradientes:**  
  La memoria requerida para el forward y backward pass se reduce proporcionalmente al factor $ \frac{M_s}{M_t} $.

- **Inferencia más rápida:**  
  Un modelo con menos parámetros no solo requiere menos memoria, sino que también necesita menos operaciones de cómputo durante la inferencia. Esto se traduce en tiempos de respuesta más rápidos y en una menor demanda de recursos computacionales, lo cual es fundamental para la implementación en dispositivos con recursos limitados.

A modo de ejemplo, supongamos que cada parámetro ocupa 4 bytes. Podemos definir una función para estimar la memoria:

In [ ]:
def estimar_memoria(M, B, factor_activacion=1.0, bytes_por_param=4):
    """
    Estima la memoria necesaria para un modelo con M parámetros y un minibatch de tamaño B.
    
    Args:
        M (int): Número de parámetros del modelo.
        B (int): Tamaño del minibatch.
        factor_activacion (float): Factor para estimar el tamaño relativo de las activaciones respecto a M.
        bytes_por_param (int): Bytes por parámetro (por defecto 4 para 32 bits).
        
    Returns:
        dict: Memoria estimada en bytes para parámetros, activaciones (forward) y backward.
    """
    mem_parametros = M * bytes_por_param
    mem_forward = B * factor_activacion * M * bytes_por_param
    mem_backward = 2 * mem_forward  # Aproximación: doble de activaciones (para activaciones y gradientes)
    
    return {
        "parametros": mem_parametros,
        "forward": mem_forward,
        "backward": mem_backward
    }

# Ejemplo:
Ms = 10**6      # Modelo student con 1 millón de parámetros
Mt = 10**8      # Modelo teacher con 100 millones de parámetros
B = 32          # Tamaño del minibatch

mem_student = estimar_memoria(Ms, B)
mem_teacher = estimar_memoria(Mt, B)

print("Memoria estimada para el modelo student:")
print(mem_student)
print("\nMemoria estimada para el modelo teacher:")
print(mem_teacher)


En este ejemplo se puede observar cómo la memoria para el modelo teacher es sustancialmente mayor que para el student, tanto en el paso forward como en el paso backward.

> Nota: La memoria requerida durante el paso forward  es proporcional al tamaño del minibatch $B$ y al número de parámetros $M$ del modelo, es decir, aproximadamente $B \times \alpha \, M$. 
Durante el paso backward  se requiere almacenar además los gradientes, lo que puede duplicar la memoria necesaria. Comparando un modelo teacher con $M_t$ parámetros con un modelo student de $M_s$ parámetros, la reducción de memoria es aproximadamente proporcional a $\frac{M_s}{M_t}$, lo que repercute en inferencias más rápidas y en una menor demanda de recursos computacionales.

### **Pregunta 8**

#### **Diseño del modelo Seq2Seq**

Un modelo **Seq2Seq** clásico para traducción (o tareas similares) consta de dos módulos:

- **Encoder:** Una red recurrente (por ejemplo, LSTM) que procesa la secuencia de entrada y genera una serie de estados ocultos.  
- **Decoder:** Otra red recurrente que, paso a paso, genera la secuencia de salida. En cada paso se utiliza la información del *encoder* para mejorar la generación.

El *encoder* transforma la entrada $ \mathbf{x} = (x_1, x_2, \dots, x_{T_{enc}}) $ en una secuencia de estados ocultos:
$$
\{h_1, h_2, \dots, h_{T_{enc}}\}.
$$
El *decoder* genera la secuencia de salida $ \mathbf{y} = (y_1, y_2, \dots, y_{T_{dec}}) $ utilizando su estado oculto y, en este caso, un **mecanismo de atención**.

### 2. Implementación del mecanismo de atención

Entre los esquemas más conocidos se encuentran el de **Luong** y el de **Bahdanau**. En este ejemplo se usa la **atención de Luong** (global) con la variante *general*.

#### Fórmulas de cálculo

- **Puntuación (score) de atención:**  
  Para cada paso de decodificación $t$ y cada estado del encoder $ h_i $, se calcula:
  $$
  \text{score}(s_t, h_i) = s_t^\top W_a h_i,
  $$
  donde:
  - $ s_t $ es el estado oculto del decoder en el paso $ t $.
  - $ h_i $ es el estado oculto del encoder en la posición $ i $.
  - $ W_a $ es una matriz de parámetros a aprender.

- **Normalización (distribución atencional):**  
  Se aplica la función *softmax* a los scores para obtener los pesos:
  $$
  \alpha_{t,i} = \frac{\exp\left(\text{score}(s_t, h_i)\right)}{\sum_{j=1}^{T_{enc}} \exp\left(\text{score}(s_t, h_j)\right)}.
  $$

- **Vector de contexto:**  
  Se calcula como la suma ponderada de los estados del encoder:
  $$
  c_t = \sum_{i=1}^{T_{enc}} \alpha_{t,i} h_i.
  $$

- **Integración en el decodificador:**  
  El vector de contexto se concatena con el estado del decoder (o con el embedding de la entrada) para generar la predicción. Una forma común es:
  $$
  \tilde{s_t} = \tanh(W_c [c_t; s_t]),
  $$
  y la predicción final se obtiene aplicando una capa lineal y *softmax*:
  $$
  y_t = \text{softmax}(W_o \tilde{s_t}).
  $$


El siguiente código ilustra un modelo Seq2Seq con atención de Luong:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Encoder ---
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
    
    def forward(self, src):
        # src: (src_len, batch_size)
        embedded = self.embedding(src)  # (src_len, batch_size, emb_dim)
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs: (src_len, batch_size, hidden_dim)
        return outputs, hidden, cell

# --- Atención de Luong ---
class LuongAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(LuongAttention, self).__init__()
        # Usamos una proyección lineal para la versión "general"
        self.attn = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, decoder_hidden, encoder_outputs):
        # decoder_hidden: (1, batch_size, hidden_dim)
        # encoder_outputs: (src_len, batch_size, hidden_dim)
        # Transformamos los estados del encoder
        encoder_transformed = self.attn(encoder_outputs)  # (src_len, batch_size, hidden_dim)
        # Preparamos el estado del decoder
        decoder_hidden = decoder_hidden.squeeze(0)  # (batch_size, hidden_dim)
        # Transponemos encoder_transformed para bmm
        encoder_transformed = encoder_transformed.permute(1, 2, 0)  # (batch_size, hidden_dim, src_len)
        # Reshape del estado del decoder para bmm: (batch_size, 1, hidden_dim)
        decoder_hidden = decoder_hidden.unsqueeze(1)
        # Cálculo de scores: (batch_size, 1, src_len)
        scores = torch.bmm(decoder_hidden, encoder_transformed).squeeze(1)  # (batch_size, src_len)
        
        # Normalización para obtener distribución atencional
        attn_weights = F.softmax(scores, dim=1)  # (batch_size, src_len)
        
        # Cálculo del vector de contexto: suma ponderada de los estados del encoder
        encoder_outputs = encoder_outputs.permute(1, 0, 1)  # (batch_size, src_len, hidden_dim)
        attn_weights = attn_weights.unsqueeze(1)  # (batch_size, 1, src_len)
        context = torch.bmm(attn_weights, encoder_outputs).squeeze(1)  # (batch_size, hidden_dim)
        
        return context, attn_weights

# --- Decoder ---
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers=1, dropout=0.5):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim + hidden_dim, hidden_dim, n_layers, dropout=dropout)
        self.attention = LuongAttention(hidden_dim)
        # La capa final combina el estado del decoder y el contexto
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
    
    def forward(self, input, hidden, cell, encoder_outputs):
        # input: (batch_size) token de entrada en el paso actual
        input = input.unsqueeze(0)  # (1, batch_size)
        embedded = self.embedding(input)  # (1, batch_size, emb_dim)
        
        # Se calcula la atención usando el último estado del decoder
        context, attn_weights = self.attention(hidden[-1].unsqueeze(0), encoder_outputs)
        context = context.unsqueeze(0)  # (1, batch_size, hidden_dim)
        
        # Concatenamos el embedding y el vector de contexto
        lstm_input = torch.cat((embedded, context), dim=2)  # (1, batch_size, emb_dim + hidden_dim)
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
        
        # Combinamos la salida del decoder y el contexto para predecir la siguiente palabra
        output = output.squeeze(0)   # (batch_size, hidden_dim)
        context = context.squeeze(0) # (batch_size, hidden_dim)
        prediction = self.fc_out(torch.cat((output, context), dim=1))  # (batch_size, output_dim)
        
        return prediction, hidden, cell, attn_weights


**Proceso de decodificación con atención**

>1. **Entrada del token:** Se toma el token actual (ya sea la palabra real en *teacher forcing* o la generada en pasos previos).
>2. **Embedding:** Se obtiene su representación embebida.
>3. **Cálculo de atención:**  
    - Se utiliza el último estado del decoder $ s_t $ para calcular los scores con cada estado $ h_i $ del encoder mediante:
     $$
     \text{score}(s_t, h_i) = s_t^\top W_a h_i.
     $$
    - Se normalizan estos scores con softmax:
     $$
     \alpha_{t,i} = \frac{\exp(\text{score}(s_t, h_i))}{\sum_{j=1}^{T_{enc}} \exp(\text{score}(s_t, h_j))}.
     $$
    - Se obtiene el vector de contexto:
     $$
     c_t = \sum_{i=1}^{T_{enc}} \alpha_{t,i} h_i.
     $$
> 4. **Integración en el LSTM del decoder:**  
   Se concatena el embedding del token y $ c_t $ para alimentar la LSTM y actualizar el estado.
>5. **Predicción:** Finalmente, se combina la salida del decoder y el contexto para predecir el siguiente token.

#### Reducción del desvanecimiento de gradiente

El mecanismo de atención ayuda a mitigar el **desvanecimiento de gradiente** en redes recurrentes largas cuando:

- **Secuencias largas:** Al permitir al decoder acceder directamente a cualquier estado del encoder, se evita la dependencia exclusiva del estado final del encoder. Esto facilita que el gradiente fluya a lo largo de la secuencia, mejorando la propagación del error en pasos lejanos.

#### Ventajas en tareas multimodales

1. **Focalización selectiva:**  
   En tareas como la descripción de imágenes, la atención permite al modelo concentrarse en regiones específicas de la imagen relevantes para cada palabra generada, mejorando la coherencia y relevancia de la descripción.

2. **Interpretabilidad:**  
   Los pesos de atención ofrecen una pista sobre qué partes de la entrada (por ejemplo, regiones de la imagen) influyen en la salida, lo que permite visualizar y entender el proceso de toma de decisiones del modelo.

#### Limitación concreta

- **Complejidad computacional:**  
  En tareas multimodales con entradas de alta dimensión (como imágenes de alta resolución o videos), el cálculo de la atención (especialmente si se realiza de forma global) puede resultar costoso en términos de tiempo de cómputo y memoria. Esto puede dificultar la implementación en entornos con recursos limitados o en aplicaciones en tiempo real.
